## 一、导入库

In [12]:
import numpy as np
from dataclasses import dataclass
from typing import List, Tuple
import random
import scipy.optimize as opt
import scipy.linalg as sla
from sklearn.model_selection import StratifiedKFold

## 二、数据读取与处理

In [2]:
# 数据加载（复用作业给的风格）
@dataclass
class Dataset:
    Xtr: np.ndarray  # noisy train+val 的特征
    Str: np.ndarray  # noisy train+val 的标签（{0,1,2}）
    Xts: np.ndarray  # clean test 的特征
    Yts: np.ndarray  # clean test 的标签（{0,1,2}）

def load_npz(path): 
    d=np.load(path); 
    return Dataset(d['Xtr'],d['Str'],d['Xts'],d['Yts'])

#其他函数定义
def set_seed(seed: int):
    np.random.seed(seed)
    random.seed(seed)

def one_hot(y: np.ndarray, C: int) -> np.ndarray:
    oh = np.zeros((y.shape[0], C), dtype=np.float32)
    oh[np.arange(y.shape[0]), y] = 1.0
    return oh

def softmax(z: np.ndarray) -> np.ndarray:
    # 数值稳定
    z = z - z.max(axis=1, keepdims=True)
    expz = np.exp(z)
    return expz / (expz.sum(axis=1, keepdims=True) + 1e-12)

def accuracy(y, yhat): return float((y==yhat).mean())


#根据文件名返回已知的转移矩阵 T。
def get_T(name: str) -> np.ndarray:
    name = name.lower()
    if "0.3" in name:
        T = np.array([[0.7, 0.3, 0.0],
                      [0.0, 0.7, 0.3],
                      [0.3, 0.0, 0.7]], dtype=np.float32)
    elif "0.6" in name:
        T = np.array([[0.4, 0.3, 0.3],
                      [0.3, 0.4, 0.3],
                      [0.3, 0.3, 0.4]], dtype=np.float32)
    else:
        raise ValueError("Unknown dataset: only 0.3 and 0.6 are supported here.")
    return T

# 训练集划分
def split(X,y,ratio=0.2,seed=42):
    set_seed(seed); 
    n=X.shape[0]; 
    idx=np.random.permutation(n); 
    sp=int(n*(1-ratio))
    return X[idx[:sp]],y[idx[:sp]],X[idx[sp:]],y[idx[sp:]]

## 三、ccn算法实现

### prepocess

In [3]:
@dataclass
class Standardizer:
    mean: np.ndarray; std: np.ndarray
    def transform(self,Xf):
        Xn=(Xf-self.mean)/self.std
        bias=np.ones((Xn.shape[0],1),dtype=np.float64)
        return np.hstack([Xn,bias])

def flatten(X): 
    return X.reshape(X.shape[0],-1).astype(np.float64)
def fit_std(Xtrf): 
    m=Xtrf.mean(0,keepdims=True); 
    s=Xtrf.std(0,keepdims=True)+1e-5; 
    return Standardizer(m,s)

### forward方法

Forward 方法：把「干净预测分布」p_clean 乘以 T^T，得到 noisy 标签的预测分布：p_tilde = T^T * p_clean，然后对 noisy 标签做交叉熵。

In [4]:
def forward_loss(p_clean, y, T):
    p_tilde = p_clean @ T         
    p_tilde = np.clip(p_tilde, 1e-12, 1.0)
    return float(-np.log(p_tilde[np.arange(y.shape[0]), y]).mean())

返回 dL/dz（对logits的梯度）。便于与线性层合成 dL/dW = X^T @ dL/dz

In [5]:
def dLdz_forward(p_clean, y, T):
    N, C = p_clean.shape
    Y = one_hot(y, C)
    p_tilde = p_clean @ T
    p_tilde = np.clip(p_tilde, 1e-12, 1.0)
    dL_dp_tilde = -(Y / p_tilde) / N
    dL_dp_clean = dL_dp_tilde @ T.T
    s = (dL_dp_clean * p_clean).sum(axis=1, keepdims=True)
    dL_dz = p_clean * (dL_dp_clean - s)
    return dL_dz

### 多类逻辑回归模型

In [6]:
@dataclass
class FwdCfg: wd:float=1e-4; max_iter:int=300; seed:int=42
class SoftmaxFwd:
    def __init__(self, D, C, T, cfg:FwdCfg):
        self.D, self.C, self.T, self.cfg = D, C, T, cfg
        set_seed(cfg.seed)
        self.W = (0.01*np.random.randn(D,C)).astype(np.float64)
    def _fun(self,w,X,y):
        W=w.reshape(self.D,self.C); p=softmax(X@W)
        base=forward_loss(p,y,self.T)
        reg =0.5*self.cfg.wd*(sla.norm(W,'fro')**2)
        dLdz=dLdz_forward(p,y,self.T)
        grad = X.T@dLdz + self.cfg.wd*W
        return base+reg, grad.reshape(-1)
    def fit(self,X,y):
        res=opt.minimize(self._fun,self.W.reshape(-1),args=(X,y),method="L-BFGS-B",jac=True,
                         options={"maxiter":self.cfg.max_iter})  # 不再传 disp，避免 warning
        self.W=res.x.reshape(self.D,self.C); return res
    def predict_proba(self,X): return softmax(X@self.W)
    def predict(self,X): return self.predict_proba(X).argmax(1)
       

### 调参

In [13]:
@dataclass
class Result:
    cfg: "FwdCfg"
    val_loss: float
    test_acc: float

#10折交叉验证
def search_with_direction_cv(Xtr, Str, Xts, Yts, T, seed=42, n_splits=10) -> Tuple[Result, List[Result]]:
    # 展平
    Xtr_f = flatten(Xtr)
    Xts_f = flatten(Xts)
    C = T.shape[0]
    D = Xtr_f.shape[1]

    # 超参网格（可按需改）
    grid = [
        FwdCfg(wd=1e-4, max_iter=500, seed=seed),
        FwdCfg(wd=1e-3, max_iter=500, seed=seed),
        FwdCfg(wd=1e-2, max_iter=500, seed=seed),
        FwdCfg(wd=5e-2, max_iter=500, seed=seed),
        FwdCfg(wd=1e-1, max_iter=500, seed=seed),
        FwdCfg(wd=5e-1, max_iter=500, seed=seed),
    ]

    # 分层 10 折（对噪声标签 Str 分层即可）
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)

    cfg_cv_losses = []   # [(cfg, mean_val_loss)]
    for cfg in grid:
        fold_losses = []
        for tr_idx, val_idx in skf.split(Xtr_f, Str):
            X_tr_raw, y_tr = Xtr_f[tr_idx], Str[tr_idx]
            X_val_raw, y_val = Xtr_f[val_idx], Str[val_idx]

            # 只用本折训练部分拟合标准化器，避免泄露
            std = fit_std(X_tr_raw)
            X_tr = std.transform(X_tr_raw)
            X_val = std.transform(X_val_raw)

            # 训练并在该折验证
            model = SoftmaxFwd(D, C, T, cfg)
            model.fit(X_tr, y_tr)

            p_val = model.predict_proba(X_val)
            vloss = forward_loss(p_val, y_val, T) + 0.5 * cfg.wd * (sla.norm(model.W, 'fro') ** 2)
            fold_losses.append(vloss)

        cfg_cv_losses.append((cfg, float(np.mean(fold_losses))))

    # 选平均验证损失最小的 cfg
    best_cfg, best_cv_loss = min(cfg_cv_losses, key=lambda x: x[1])

    std_full = fit_std(Xtr_f)
    X_tr_full = std_full.transform(Xtr_f)
    X_te = std_full.transform(Xts_f)

    best_model = SoftmaxFwd(D, C, T, best_cfg)
    best_model.fit(X_tr_full, Str)

    test_acc = accuracy(Yts, best_model.predict(X_te))

    results: List[Result] = [
        Result(cfg=cfg, val_loss=mean_vloss, test_acc=test_acc if cfg is best_cfg else np.nan)
        for cfg, mean_vloss in cfg_cv_losses
    ]

    best = Result(cfg=best_cfg, val_loss=best_cv_loss, test_acc=test_acc)
    return best, results

## 四、main部分

In [14]:
def main():
    set_seed(42)
    datasets = [
        "datasets/FashionMNIST0.3.npz",
        "datasets/FashionMNIST0.6.npz",
    ]
    for path in datasets:
        print(f"\n==== Dataset: {path} ====")
        T = get_T(path)
        data = load_npz(path)
        best, results = search_with_direction(data.Xtr, data.Str, data.Xts, data.Yts, T, seed=42)

        def fmt(r:Result):
            return f"[p@T] wd={r.cfg.wd}, it={r.cfg.max_iter} | val_loss={r.val_loss:.4f}, test_acc={r.test_acc*100:.2f}%"

        print("Tuned configs (both orientations):")
        for r in results: print(" ", fmt(r))
        print("** Best:", fmt(best))

if __name__ == "__main__":
    main()


==== Dataset: datasets/FashionMNIST0.3.npz ====
Tuned configs (both orientations):
  [p@T] wd=0.0001, it=500 | val_loss=0.7299, test_acc=95.47%
  [p@T] wd=0.001, it=500 | val_loss=0.6817, test_acc=96.23%
  [p@T] wd=0.05, it=500 | val_loss=0.6749, test_acc=97.07%
  [p@T] wd=0.01, it=500 | val_loss=0.6633, test_acc=97.47%
  [p@T] wd=0.5, it=500 | val_loss=0.7295, test_acc=96.30%
  [p@T] wd=0.1, it=500 | val_loss=0.6854, test_acc=96.93%
** Best: [p@T] wd=0.01, it=500 | val_loss=0.6633, test_acc=97.47%

==== Dataset: datasets/FashionMNIST0.6.npz ====
Tuned configs (both orientations):
  [p@T] wd=0.0001, it=500 | val_loss=1.1009, test_acc=72.17%
  [p@T] wd=0.001, it=500 | val_loss=1.0996, test_acc=72.77%
  [p@T] wd=0.05, it=500 | val_loss=1.0935, test_acc=95.13%
  [p@T] wd=0.01, it=500 | val_loss=1.0943, test_acc=92.10%
  [p@T] wd=0.5, it=500 | val_loss=1.0964, test_acc=92.83%
  [p@T] wd=0.1, it=500 | val_loss=1.0941, test_acc=94.97%
** Best: [p@T] wd=0.05, it=500 | val_loss=1.0935, test_a

### 以下是用最佳wd以及东晴提供的转移矩阵来跑CIFAR数据集的结果

In [16]:
# ==== CIFAR (ForwardCorrection, softmax) - 10-fold CV on wd (fixed dims) ====

# 已给的 T 与数据路径
T_CIFAR = np.array([[0.3983,0.332 ,0.2697],
                    [0.2967,0.3587,0.3447],
                    [0.31  ,0.3097,0.3803]], dtype=np.float64)
DATASET_PATH = "datasets/CIFAR.npz"

# 载入
d = np.load(DATASET_PATH)
Xtr, Str, Xts, Yts = d['Xtr'], d['Str'], d['Xts'], d['Yts']

# 展平（不要先做固定切分）
Xtr_f = flatten(Xtr)
Xts_f = flatten(Xts)

@dataclass
class FwdCfg:
    wd: float = 0.05
    max_iter: int = 500
    seed: int = 42

C = T_CIFAR.shape[0]
seed = 42

# 待测试的 wd 列表（沿用你原来的顺序）
grid = [
    FwdCfg(wd=1e-4, max_iter=500, seed=seed),
    FwdCfg(wd=1e-3, max_iter=500, seed=seed),
    FwdCfg(wd=5e-2, max_iter=500, seed=seed),
    FwdCfg(wd=1e-2, max_iter=500, seed=seed),
    FwdCfg(wd=5e-1, max_iter=500, seed=seed),
    FwdCfg(wd=1e-1, max_iter=500, seed=seed),
    FwdCfg(wd=0.2,   max_iter=500, seed=seed),
    FwdCfg(wd=0.5,   max_iter=500, seed=seed),
]

print(f"\n==== Dataset: {DATASET_PATH} ====")
print("Tuned configs with 10-fold CV (forward loss):")

# 10 折分层（按噪声标签 Str 分层）
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

# 逐 cfg 做交叉验证
cv_summaries = []  # [(cfg, mean_vloss, std_vloss)]
for cfg in grid:
    fold_losses = []
    for tr_idx, val_idx in skf.split(Xtr_f, Str):
        # 划分本折数据
        X_tr_raw, y_tr = Xtr_f[tr_idx], Str[tr_idx]
        X_val_raw, y_val = Xtr_f[val_idx], Str[val_idx]

        std = fit_std(X_tr_raw)
        X_tr = std.transform(X_tr_raw)
        X_val = std.transform(X_val_raw)

        D_fold = X_tr.shape[1]

        # 训练并在该折验证
        model = SoftmaxFwd(D_fold, C, T_CIFAR, cfg)
        res = model.fit(X_tr, y_tr)

        p_val = model.predict_proba(X_val)
        vloss = forward_loss(p_val, y_val, T_CIFAR) + 0.5 * cfg.wd * (sla.norm(model.W, 'fro')**2)
        fold_losses.append(float(vloss))

    mean_vloss = float(np.mean(fold_losses))
    std_vloss  = float(np.std(fold_losses, ddof=1)) if len(fold_losses) > 1 else 0.0
    cv_summaries.append((cfg, mean_vloss, std_vloss))
    print(f"  [p@T] wd={cfg.wd:g}, it={cfg.max_iter} | mean_val_loss={mean_vloss:.4f} (±{std_vloss:.4f})")

# 选平均验证损失最小的 cfg
best_cfg, best_mean_vloss, best_std = min(cv_summaries, key=lambda x: x[1])

std_full = fit_std(Xtr_f)
X_tr_full = std_full.transform(Xtr_f)
X_te      = std_full.transform(Xts_f)

D_full = X_tr_full.shape[1]
best_model = SoftmaxFwd(D_full, C, T_CIFAR, best_cfg)
res_full = best_model.fit(X_tr_full, Str)

# 测试集准确率（clean）
test_acc = float((best_model.predict(X_te) == Yts).mean())

# 汇总 results
results = []
for cfg, mean_vloss, std_vloss in cv_summaries:
    results.append({
        "wd": cfg.wd,
        "iters": getattr(res_full, "nit", "N/A") if cfg is best_cfg else "N/A",
        "val_loss": mean_vloss,         # 10 折平均验证损失
        "val_loss_std": std_vloss,      # 波动
        "test_acc": test_acc if cfg is best_cfg else float('nan'),
    })

# 打印最优
print(f"** Best (by 10-fold mean val loss): [p@T] wd={best_cfg.wd:g}, it={best_cfg.max_iter} | "
      f"mean_val_loss={best_mean_vloss:.4f} (±{best_std:.4f}), test_acc={test_acc*100:.2f}%")



==== Dataset: datasets/CIFAR.npz ====
Tuned configs with 10-fold CV (forward loss):
  [p@T] wd=0.0001, it=500 | mean_val_loss=1.1006 (±0.0027)
  [p@T] wd=0.001, it=500 | mean_val_loss=1.1042 (±0.0019)
  [p@T] wd=0.05, it=500 | mean_val_loss=1.0962 (±0.0015)
  [p@T] wd=0.01, it=500 | mean_val_loss=1.0985 (±0.0018)
  [p@T] wd=0.5, it=500 | mean_val_loss=1.0966 (±0.0011)
  [p@T] wd=0.1, it=500 | mean_val_loss=1.0961 (±0.0014)
  [p@T] wd=0.2, it=500 | mean_val_loss=1.0962 (±0.0013)
  [p@T] wd=0.5, it=500 | mean_val_loss=1.0966 (±0.0011)
** Best (by 10-fold mean val loss): [p@T] wd=0.1, it=500 | mean_val_loss=1.0961 (±0.0014), test_acc=66.67%
